In [14]:
import numpy as np 
import math
import re
from math import sqrt

In [3]:
x= ([1,2,3,4,5])
y= ([2,4,6,8,10])
correlation = np.corrcoef(x,y)[0,1]
print(correlation)

0.9999999999999999


In [5]:
mean_x = sum(x)/len(x)
mean_y = sum(y)/len(y)
numerator = sum((x[i] - mean_x)*(y[i]-mean_y) for i in range(len(x)))
denominator_x = sum((x[i]-mean_x)**2 for i in range(len(x)))
denominator_y = sum((y[i]-mean_y)**2 for i in range(len(y)))
correlation1 = numerator / (denominator_x**0.5 * denominator_y**0.5)

print(correlation1)

0.9999999999999998


Retrieval văn bản với TF-IDF + Pearson (không dùng thư viện) Cho các
văn bản:

In [47]:
class TextProcess:
    def __init__(self,documents,query):
        self.documents = documents
        self.processed_documents = [self.preprocess_text(doc) for doc in documents]
        self.tf_values = [self.compute_tf(doc) for doc in self.processed_documents]
        self.idf_values = self.compute_idf(self.processed_documents)
        self.query = query
        self.processed_query = self.preprocess_text(query)
        self.query_tf = self.compute_tf(self.processed_query)
        self.query_tfidf = self.compute_tfidf_query()


    def preprocess_text(self , text):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text.split()
    def compute_tf (self, words):
        tf={}
        total_words = len(words)
        for word in words:
            if word in tf:
                tf[word] +=1
            else:
                tf[word] = 1
        
        for word in tf:
            tf[word] = tf[word]/len(words)
        return tf
    
    def compute_idf(self,documents):
        idf = {}
        total_documents = len(documents)
        word_doc_count = {}
        for doc in documents:
            unique_words = set(doc)
            for word in unique_words:
                if word in word_doc_count:
                    word_doc_count[word] +=1
                else:
                    word_doc_count[word] = 1

        for word , count in word_doc_count.items():
            idf[word] = math.log(total_documents/(1+count))
        return idf
    
    def compute_tfidf(self):
        tfidf_matrix = []
        
        for tf in self.tf_values:  # Duyệt qua từng tài liệu (TF vector)
            tfidf_vector = {}
            for word in tf:  # Duyệt qua từng từ trong TF vector của tài liệu
                # Tính TF-IDF trực tiếp
                tf_value = tf[word]
                # Lấy IDF của từ từ idf_values (không dùng get)
                idf_value = self.idf_values[word] if word in self.idf_values else 0
                tfidf_vector[word] = tf_value * idf_value
        
            tfidf_matrix.append(tfidf_vector)
        return tfidf_matrix

        
    
    def compute_tfidf_query(self):
        query_tfidf = {}
        for word in self.query_tf:
            tf_value = self.query_tf[word]
            idf_value = self.idf_values[word] if word in self.idf_values else 0
            query_tfidf[word] = tf_value * idf_value
        return query_tfidf

    def pearson_correlation(self, vec1, vec2):
        # Chắc chắn rằng vec1 và vec2 là các từ điển
        common_words = [word for word in vec1.keys() if word in vec2]
        
        if len(common_words) == 0:
            return 0  # Không có từ chung, trả về hệ số tương quan bằng 0
        
        sum_xy = 0
        sum_x = 0
        sum_y = 0
        sum_x2 = 0
        sum_y2 = 0

        for word in common_words:
            x = vec1[word]  # Truy xuất giá trị từ vec1 (TF hoặc TF-IDF)
            y = vec2[word]  # Truy xuất giá trị từ vec2 (TF hoặc TF-IDF)
            sum_xy += x * y
            sum_x += x
            sum_y += y
            sum_x2 += x ** 2
            sum_y2 += y ** 2

        n = len(common_words)
        
        # Sửa lại công thức tính hệ số tương quan
        numerator = (n * sum_xy) - (sum_x * sum_y)
        denominator = math.sqrt((n * sum_x2 - sum_x**2) * (n * sum_y2 - sum_y**2))

        if denominator == 0:
            return 0  # Tránh chia cho 0
        
        return numerator / denominator





    def rank_documents(self):
        tfidf_matrix = self.compute_tfidf()  # Chỉ lấy tfidf_matrix
        correlations = []

        for i, doc_tfidf in enumerate(tfidf_matrix):
            correlation = self.pearson_correlation(doc_tfidf, self.query_tfidf)  # Đảm bảo query_tfidf đã có giá trị
            correlations.append((i, correlation))

        correlations.sort(key=lambda x: x[1], reverse=True)
        return correlations



    

In [48]:
doc1 = "deep learning for natural language processing"
doc2 = "transformer models improve language understanding"
doc3 = "convolutional neural networks for image classification"
query = "language models for text understanding"

documents = [doc1, doc2, doc3]
text_processor = TextProcess(documents=documents, query=query)

# Xếp hạng các văn bản theo mức độ liên quan
ranked_docs = text_processor.rank_documents()

# In ra kết quả
print("Ranked Documents (Index, Pearson Correlation):")
for doc_index, correlation in ranked_docs:
    print(f"Document {doc_index+1}: {correlation}")

Ranked Documents (Index, Pearson Correlation):
Document 2: 1.0
Document 1: 0
Document 3: 0
